In [11]:
import os, sys
import numpy as np
from collections import OrderedDict as OD
#import uproot3 as uproot
#import uproot3 
import uproot as uproot
import hist
import matplotlib.pyplot as plt
import mplhep as hep
import json

sys.path.insert(1, '../') # to import file from other directory (../ in this case)

from htoaa_CommonTools import(
    getTH1BinContent
)

#sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20230831_SelPNetMDXbbNSV/2018/analyze_htoaa_stage1.root'
#sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20230831_SelPNetMDXbbNSV/2018/plots'
#sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20230914_PNetMDXbbNSV_wSFPNetMDXbb_1/2018/analyze_htoaa_stage1.root'
#sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20230914_PNetMDXbbNSV_wSFPNetMDXbb_1/2018/plots'
#sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20230921_SFPNetMDXbbvsQCD/2018/analyze_htoaa_stage1.root'
#sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20230921_SFPNetMDXbbvsQCD/2018/plots'
sIpFile = '/eos/cms/store/user/ssawant/htoaa/analysis/20240806_VBFjj_Sig/2018/analyze_htoaa_stage1.root'
sOpDir  = '/eos/cms/store/user/ssawant/htoaa/analysis/20240806_VBFjj_Sig/2018/plots'

CAT = 'VBFjj' # 'gg0l', 'VBFjj', 'Wlv', 'Zll', 'Zvv',  'Vjj'. 'ZvvIncl','ZvvLo', 'ZvvHi', 'gg0lIncl', 'gg0lLo', 'gg0lHi', 'tt0l', 'VBFjj'
# 'tt0l_ge1NonHFatJet_0BExtra', 'tt0l_ge1NonHFatJet_1BExtra', 'tt0l_ge1NonHFatJet_ge2BExtra', 'tt0l_0NonHFatJet_ge2B'

sSubcategory = '%s_' % (CAT) if CAT in ['ZvvIncl', 'ZvvLo', 'ZvvHi', 'gg0lIncl', 'gg0lLo', 'gg0lHi', 'tt0l_ge1NonHFatJet_0BExtra', 'VBFjj'] else '' # 'ZvvIncl' # 'gg0l', 'VBFjj', 'Wlv', 'Zll', 'Zvv',  'Vjj'
#selectionTags = ['%sSBWP80to40' % (sSubcategory)]
#selectionTags = ['tt0l_ge1NonHFatJet_0BExtra_SRWP80', 'tt0l_ge1NonHFatJet_1BExtra_SRWP80', 'tt0l_ge1NonHFatJet_ge2BExtra_SRWP80', 'tt0l_0NonHFatJet_ge2B_SRWP80']
#selectionTags = [ 'tt0l_ge1NonHFatJet_1BExtra_Hi_SRWP80', 'tt0l_ge1NonHFatJet_1BExtra_Med_SRWP80', 'tt0l_ge1NonHFatJet_1BExtra_Lo_SRWP80' ]
selectionTags = ['gg0lIncl_SRWP40', 'VBFLoose_SRWP40', 'VBFTight_SRWP40', 'VBFLoose_SRWP80', 'VBFTight_SRWP80']

sOpDir = '%s/%s' % (sOpDir, CAT)
if not os.path.exists(sOpDir):
    os.makedirs(sOpDir)
    
fIpFile = uproot.open(sIpFile)

In [12]:
fIpFile['evt'].keys()

['ggHtoaato4b_mA_12;1',
 'ggHtoaato4b_mA_12/hPV_npvs_beforeSel_Nom;1',
 'ggHtoaato4b_mA_12/hPV_npvsGood_beforeSel_Nom;1',
 'ggHtoaato4b_mA_12/hGenHiggsPt_all_Nom;1',
 'ggHtoaato4b_mA_12/hGenHiggsLog2Pt_all_Nom;1',
 'ggHtoaato4b_mA_12/hGenHiggsPt_GenHToAATo4B_Nom;1',
 'ggHtoaato4b_mA_12/hGenHiggsPt_sel_Nom;1',
 'ggHtoaato4b_mA_12/hGenHiggsPt_sel_wGenCuts_Nom;1',
 'ggHtoaato4b_mA_12/hGenHiggsMass_all_0_Nom;1',
 'ggHtoaato4b_mA_12/hMass_GenA_all_0_Nom;1',
 'ggHtoaato4b_mA_12/hGenHiggsMass_all_Nom;1',
 'ggHtoaato4b_mA_12/hMass_GenA_all_Nom;1',
 'ggHtoaato4b_mA_12/hMass_GenAApair_all_Nom;1',
 'ggHtoaato4b_mA_12/hMass_GenAToBBbarpair_all_Nom;1',
 'ggHtoaato4b_mA_12/hMass_Gen4BFromHToAA_all_Nom;1',
 'ggHtoaato4b_mA_12/hMass_GenAToBBbarpair_all_1_Nom;1',
 'ggHtoaato4b_mA_12/hMass_Gen4BFromHToAA_all_1_Nom;1',
 'ggHtoaato4b_mA_12/hDeltaR_GenH_GenB_max_Nom;1',
 'ggHtoaato4b_mA_12/hMass_GenA1_vs_GenA2_all_Nom;1',
 'ggHtoaato4b_mA_12/hMass_GenA1ToBBbar_vs_GenA2ToBBbar_all_Nom;1',
 'ggHtoaato4b_mA_1

In [13]:
'''
for histName_ in fIpFile['evt/Data'].keys():
    if 'hCutFlowWeighted' in histName_:
        print(f"{histName_}")
        if 'sel_leadingFatJetParticleNetMD_XbbvsQCD' in histName_:
            print(f"{histName_} that is wanted\n\n")
'''

'\nfor histName_ in fIpFile[\'evt/Data\'].keys():\n    if \'hCutFlowWeighted\' in histName_:\n        print(f"{histName_}")\n        if \'sel_leadingFatJetParticleNetMD_XbbvsQCD\' in histName_:\n            print(f"{histName_} that is wanted\n\n")\n'

In [17]:
#sData = "Data"
ExpData_dict = {} #{
    #'Data': ['JetHT_Run2018A', 'JetHT_Run2018B', 'JetHT_Run2018C', 'JetHT_Run2018D']
    #'Data': ['MET_Run2018A', 'MET_Run2018B', 'MET_Run2018C', 'MET_Run2018D'] if CAT in ['ZvvIncl'] else ['JetHT_Run2018A', 'JetHT_Run2018B', 'JetHT_Run2018C', 'JetHT_Run2018D'] # 'gg0l', 'VBFjj', 'Wlv', 'Zll', 'Zvv',  'Vjj'
    #'Data ABCD': ['SingleMuon_Run2018A', 'SingleMuon_Run2018B', 'SingleMuon_Run2018C', 'SingleMuon_Run2018D'],
#}
ExpData_dict['Data'] = []
#ExpData_dict['Data'] = ['JetHT_Run2018A', 'JetHT_Run2018B', 'JetHT_Run2018C', 'JetHT_Run2018D']
if 'Zvv' in CAT:
    ExpData_dict['Data'] = ['MET_Run2018A', 'MET_Run2018B', 'MET_Run2018C', 'MET_Run2018D']

#MCBkg_list = ['QCD_bEnrich', 'QCD_bGen', 'QCD_Incl', 'TTJets_NLO', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
#MCBkg_list = ['QCD',  'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
#MCBkg_list = ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat',  'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']

#MCBkg_list = ['QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat',  'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', 'WJetsToQQ_HT', 'ZJetsToQQ_HT']
#MCBkg_list = [
#    'QCD_0bCat', 'QCD_1bCat', 'QCD_2bCat', 'QCD_3bCat', 'QCD_4bCat', 'QCD_5bAndMoreCat',  
#    'TTToHadronic_powheg', 'TTToSemiLeptonic_powheg', 'TTTo2L2Nu_powheg', "SingleTop", 
#    'ZJetsToQQ_HT', "ZJetsToNuNuQ_HT",
#    "DYJets_M-50_Incl_NLO", #"DYJets_M-10to50_Incl_NLO", "DYJets_M-50_Incl_NLO", 
#    'WJetsToQQ_HT', 
#    'WJetsToLNu_HT_LO', #'WJetsToLNu_Incl_NLO', 
#    'ZZ','WZ','WW', 'ZZZ','WZZ','WWZ','WWW',
#    'GluGluHToBB_Pt-200ToInf', 
#    #'VBFHToBB_powheg', 'VBFHToBB_dipoleRecoilOn', 
#    #'VBFWH_HToBB_WToLNu_dipoleRecoilOn', 
#    'WH_HToBB_WToQQ', 'WH_HToBB_WToLNu', 
#    'ZH_HToBB', #'ggZH_HToBB', 
#    'ttH', 
#]
MCBkg_list = [
    "QCD_bEnr", "QCD_BGen", "QCD_Incl", 
    "TT0l", "TT1l", "TT2l", 
    "STop_t", "STbar_t", "ST_s_0l", "ST_s_1l", "STop_tW_Incl", "STbar_tW_Incl", "STop_tW_12l", "STbar_tW_12l", 
    "ttZ", "ttW", "tZq", 
    "Zqq", "Zvv", "Zll", "Wqq", "Wlv", 
    "ZZ", "WZ", "WW", "ZZZ", "WZZ", "WWZ", "WWW", 
    "ggH", #"VBFH", "VBFH_dipoleRecoilOn", "VBFWH_dipoleRecoilOn", 
    "WHbbqq", "WHbblv", 
    "ZH", #"ggZH", 
    "ttH", #"", "", "", 
    #"", "", "", "", "", "",     
]
MCBkg_list = []

MCSig_list = [
    #'SUSY_GluGluH_01J_HToAATo4B_M-15_HPtAbv150', 
    #'SUSY_GluGluH_01J_HToAATo4B_M-30_HPtAbv150', 
    #'SUSY_GluGluH_01J_HToAATo4B_M-55_HPtAbv150', 
    #'SUSY_ZH_ZToAll_HToAATo4B_M-15_HPtAbv150',
    #'SUSY_ZH_ZToAll_HToAATo4B_M-30_HPtAbv150',
    #'SUSY_ZH_ZToAll_HToAATo4B_M-55_HPtAbv150',
    'ggHtoaato4b_mA_20',
    'VBFHtoaato4b_mA_20',
    #'WHtoaato4b_mA_20',
    #'ZHtoaato4b_mA_20',
    #'ttHtoaato4b_mA_20',
    #
    'ggHtoaato4b_mA_30',
    'VBFHtoaato4b_mA_30',
    #'WHtoaato4b_mA_30',
    #'ZHtoaato4b_mA_30',
    #'ttHtoaato4b_mA_30',
]
sLableSig  = [
    #'GluGluHToAATo4B_M-15', 
    #'GluGluHToAATo4B_M-30', 
    #'GluGluHToAATo4B_M-55',
    #'ZHToAATo4B_M-15', 
    #'ZHToAATo4B_M-30', 
    #'ZHToAATo4B_M-55',
    'ggHtoaato4b_mA_20',
    'VBFHtoaato4b_mA_20',
    #'WHtoaato4b_mA_20',
    #'ZHtoaato4b_mA_20',
    #'ttHtoaato4b_mA_20',
    #
    'ggHtoaato4b_mA_30',
    'VBFHtoaato4b_mA_30',
    #'WHtoaato4b_mA_30',
    #'ZHtoaato4b_mA_30',
    #'ttHtoaato4b_mA_30',
]

systematics_list = ['Nom'] # ['central']
systematics_forData = 'noweight'
xValueForEventYield = 4
#selectionTags = ['sel_HLT_AK8PFJet330_TrimMass30_PFAK8BoostedDoubleB_np4', 'sel_leadingFatJetMSoftDrop', 'sel_leadingFatJetParticleNetMD_bbvsQCD']
#selectionTags = ['sel_HLT_AK8PFJet330_TrimMass30_PFAK8BoostedDoubleB_np4', 'sel_leadingFatJetMSoftDrop', 'sel_leadingFatJetParticleNetMD_XbbvsQCD']
#selectionTags = ['sel_HLT_AK8PFJet330_TrimMass30_PFAK8BoostedDoubleB_np4', 'sel_2018HEM1516Issue', 'sel_leadingFatJetMSoftDrop', 'sel_leadingFatJetParticleNetMD_XbbvsQCD', 'SR'] #['SR', 'sel_leadingFatJetMSoftDrop', ]
#selectionTags = ['sel_leadingMuonPt', 'sel_leadingMuonEta', 'sel_L1_SingleMu22', 'sel_HLT_IsoMu24', 'sel_dR_Muon_FatJet', 'sel_leadingFatJetEta', 'sel_JetID', 'sel_leadingFatJetZHbb_Xbb_avg', 'SR', 'SR_Trg_Combo_AK4AK8Jet_HT']
#sSubcategory = '%s_' % (CAT) if CAT in ['ZvvIncl', 'ZvvLo', 'ZvvHi', 'gg0lIncl', 'gg0lLo', 'gg0lHi'] else '' # 'ZvvIncl' # 'gg0l', 'VBFjj', 'Wlv', 'Zll', 'Zvv',  'Vjj'
#selectionTags = ['%sSBWP80to40' % (sSubcategory)]

eventYields = OD()
sEventYield = "%s \n" % (sIpFile)
MCBkg_total = 0
MCSig_total = 0
for selectionTag in selectionTags:   
    for systematics_name in systematics_list:
        sEventYield += "Selection: %s,  systematics: %s \n" % (selectionTag, systematics_name)
        sEventYield += "%-45s:  %15s +- %15s  (%15s) \n" % ('Samples', 'nEvents', 'Uncertainty', 'nEvents_unweighted')
        eventYields[systematics_name] = OD()
        MCBkg_total = 0;   MCBkg_total_variance = 0
        MCSig_total = 0;   MCSig_total_variance = 0
        Data_total  = 0;   Data_total_variance  = 0

        for sData, ExpData_list in ExpData_dict.items():
            for sample_category in MCSig_list + MCBkg_list + ExpData_list:
                # "hCutFlowWeighted"
                systematics_name_toUse = systematics_name if sample_category not in ExpData_list else systematics_forData

                histo_name_full = 'evt/%s/%s_%s_%s' % (sample_category, "hCutFlowWeighted", selectionTag, systematics_name_toUse)
                #print(f"{histo_name_full = }")
                h = fIpFile[histo_name_full].to_hist()
                nEvt_and_var = getTH1BinContent(h, xValue=xValueForEventYield)

                #histo_name_full = 'evt/%s/%s_%s' % (sample_category, "hCutFlow", systematics_name_toUse)
                histo_name_full = 'evt/%s/%s_%s_%s' % (sample_category, "hCutFlow", selectionTag, systematics_name_toUse)
                h = fIpFile[histo_name_full].to_hist()
                nEvt_and_var_unwgt = getTH1BinContent(h, xValue=xValueForEventYield)

                if sample_category in MCBkg_list:      
                    MCBkg_total          += nEvt_and_var.value
                    MCBkg_total_variance += nEvt_and_var.variance 

                if sample_category in MCSig_list:      
                    MCSig_total          += nEvt_and_var.value
                    MCSig_total_variance += nEvt_and_var.variance

                if sample_category in ExpData_list:    
                    Data_total  += nEvt_and_var.value
                    Data_total_variance  += nEvt_and_var.variance

                #print(f"{sample_category = },  \t\t\t\t\t nEvents: {nEvt_and_var.value}, \t\t  {np.sqrt(nEvt_and_var.variance)}")
                #print("%-45s:  %15g +- %15g  (%15d)" % (sample_category, nEvt_and_var.value, np.sqrt(nEvt_and_var.variance), nEvt_and_var_unwgt.value))
                sEventYield += "%-45s:  %15g +- %15g  (%15d) \n" % (sample_category, nEvt_and_var.value, np.sqrt(nEvt_and_var.variance), nEvt_and_var_unwgt.value)


                eventYields[systematics_name][sample_category] = OD([
                    ('nEvents', nEvt_and_var.value),
                    ('Uncertainty', np.sqrt(nEvt_and_var.variance)),
                    ('nEventsUnwgt', nEvt_and_var_unwgt.value),
                    ('isMCBkg', sample_category in MCBkg_list)
                    ])  
            
        sEventYield += "\n"
        sEventYield += "%-45s:  %15g +- %15g  \n" % ('Data_total', Data_total, np.sqrt(Data_total_variance))
        sEventYield += "%-45s:  %15g +- %15g  \n" % ('Bkg_total', MCBkg_total, np.sqrt(MCSig_total_variance))
        sEventYield += "%-45s:  %15g  \n" % ('Data/MCBkg', Data_total / MCBkg_total if MCBkg_total > 0 else 0)
        for idx_, sample_category in enumerate(MCSig_list):
            if abs(MCBkg_total - 0) < 1e-6: continue
            sEventYield += "%-45s:  %15g  \n" % ('S/sqrt(B) @ %s' % (sLableSig[idx_]), eventYields[systematics_name][sample_category]['nEvents'] / np.sqrt(MCBkg_total))
        sEventYield += "\n\n" 


        ## print sorted event yield table
        #bkgs_list = list(eventYields[systematics_name].keys())
        bkgs_list = [ process for process in eventYields[systematics_name].keys() if eventYields[systematics_name][process]['isMCBkg']  ]
        nEvent_bkgs_list = [ eventYields[systematics_name][bkg]['nEvents'] for bkg in bkgs_list]
        bkgRanks  = np.argsort(np.array(nEvent_bkgs_list))[::-1]
        print(f"{bkgs_list = }")
        print(f"{nEvent_bkgs_list = }")
        print(f"{np.argsort(np.array(nEvent_bkgs_list)) = }")
        print(f"{bkgRanks = }")

        sEventYield += "Sorted background processes::\n"
        for bkgRank in bkgRanks:
            processName_ = bkgs_list[bkgRank]
            sEventYield += "%-45s:  %15g +- %15g  (%15d) \n" % (
                processName_, 
                eventYields[systematics_name][processName_]['nEvents'],
                eventYields[systematics_name][processName_]['Uncertainty'],
                eventYields[systematics_name][processName_]['nEventsUnwgt'])
        sEventYield += "\n\n" 

        

print("%s" % (sEventYield))
#print(f"{json.dumps(eventYields, indent=4) = }")
#json.dumps(eventYields, indent=4)

with open('%s/eventYieldTable.txt' % (sOpDir), 'w') as fOut:
    fOut.write(sEventYield)

bkgs_list = []
nEvent_bkgs_list = []
np.argsort(np.array(nEvent_bkgs_list)) = array([], dtype=int64)
bkgRanks = array([], dtype=int64)
bkgs_list = []
nEvent_bkgs_list = []
np.argsort(np.array(nEvent_bkgs_list)) = array([], dtype=int64)
bkgRanks = array([], dtype=int64)
bkgs_list = []
nEvent_bkgs_list = []
np.argsort(np.array(nEvent_bkgs_list)) = array([], dtype=int64)
bkgRanks = array([], dtype=int64)
/eos/cms/store/user/ssawant/htoaa/analysis/20240806_VBFjj_Sig/2018/analyze_htoaa_stage1.root 
Selection: gg0lIncl_SRWP40,  systematics: Nom 
Samples                                      :          nEvents +-     Uncertainty  (nEvents_unweighted) 
ggHtoaato4b_mA_20                            :          1899.76 +-          17.133  (          14323) 
VBFHtoaato4b_mA_20                           :          671.791 +-          11.466  (           3524) 
ggHtoaato4b_mA_30                            :           1646.6 +-         15.7094  (          12885) 
VBFHtoaato4b_mA_30                     